# Pycolmap Reconstruction Options Reference
This notebook provides a detailed breakdown of the `pycolmap` options used in the 3D reconstruction pipeline, based on the configurations in `main.py`.

## 1. Feature Extraction Options
`FeatureExtractionOptions` and `SiftExtractionOptions` control how features (keypoints) are identified in each image.

| Option | Value in `main.py` | Description |
| :--- | :--- | :--- |
| `num_threads` | `8` | Number of CPU cores used for feature extraction. |
| `max_image_size` | `1600` | Images larger than this will be downscaled (saves memory/time). |
| `max_num_features` | `4000` | Maximum number of features to extract per image. |
| `first_octave` | `0` | Starting octave in the SIFT pyramid. `0` is standard, `-1` can double image size for extra features. |
| **`use_gpu`** | `False` (in `main.py`) | **CRITICAL**: Controls whether CUDA-enabled GPU is used. See Section 4. |

## 2. Feature Matching Options
`FeatureMatchingOptions` determine how features are compared between images to find correspondences.

| Option | Value in `main.py` | Description |
| :--- | :--- | :--- |
| `num_threads` | `8` | Number of CPU cores used for matching. |
| `match_sequential` | (Used in main) | Matches images in a sequence (efficient for video/ordered photos). |
| `match_exhaustive` | (Used in notebook) | Matches every image against every other image (most robust, slowest). |
| **`use_gpu`** | `False` (in `main.py`) | **CRITICAL**: GPU matching is significantly faster than CPU. |

## 3. Incremental Mapping Options
`IncrementalPipelineOptions` control the Structure from Motion (SfM) process of building the 3D model step-by-step.

| Option | Value in `main.py` | Description |
| :--- | :--- | :--- |
| `ba_global_frames_ratio` | `1.2` | Frequency of global bundle adjustment. |
| `multiple_models` | `False` | Forces the reconstruction into a single consistent 3D model. |
| `num_threads` | `8` | CPU cores used for optimization tasks. |

## 4. The Critical Importance of CUDA

While `main.py` is configured with `use_gpu = False` for compatibility, enabling CUDA is **highly recommended** for production-level 3D reconstruction.

### Why use CUDA?
1. **Performance Boost**: Feature extraction and matching can be **10x to 100x faster** on a GPU compared to a multi-core CPU.
2. **Point Density**: Because GPU matching is faster, you can afford to use higher `max_image_size` and `max_num_features` (e.g., 8000+ features). This leads to a significantly **more dense and detailed final point cloud**.
3. **Real-time processing**: For large datasets (100+ images), CPU-only reconstruction can take hours, whereas GPU-accelerated pipelines can finish in minutes.

### How to Enable
Change the following in your code:
```python
options.use_gpu = True
pycolmap.extract_features(..., device=pycolmap.Device.cuda)
```
> [!IMPORTANT]
> You must have the `pycolmap-cuda` package installed and a compatible NVIDIA GPU with drivers properly configured.